In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
DR_TYPE = "smearing" # "pca", "smearing", "none"
NDIM = 10
RANDOM_STATE = 2
SECOND_DR_TYPE = "none" # "tsne", "pca", "none"

In [ ]:
ROOT = ".."
filename = f"{ROOT}/data/hea4_dos.csv"
df = pd.read_csv(filename, index_col=None)
semicore_list = np.unique(df["semicore"].values)
n_clusters = len(semicore_list)
logdos_names = []
for i in range(100):
    logdos_names.append("log10_dos{}".format(i+1))
TARGET_NAME = 'semicore'

In [ ]:
from dos_clustering_misc import plot_each_DOS
plot_each_DOS(df, logdos_names, TARGET_NAME)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from dos_clustering_misc import add_convolution_variables
if DR_TYPE == "pca":
    _X = df[logdos_names].values
    dr = PCA(NDIM) 
    X_dr = dr.fit_transform(_X)
elif DR_TYPE == "smearing":
    df, smearedlogdos_names = add_convolution_variables(
        df, logdos_names, NDIM)
    X_dr = df[smearedlogdos_names].values
else:
    X_dr = df[logdos_names].values
    NDIM = len(logdos_names)
y = df[TARGET_NAME].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_dr = scaler.fit_transform(X_dr)


In [ ]:
from sklearn.linear_model import LogisticRegressionCV

if SECOND_DR_TYPE in ["tsne", "pca"]:
    if SECOND_DR_TYPE == "tsne":
        dr2 = TSNE(2,  init="pca", random_state=RANDOM_STATE)
    elif SECOND_DR_TYPE == "pca":
        dr2 = PCA(2, random_state=RANDOM_STATE)
    X2 = dr2.fit_transform(X_dr)

    cls = LogisticRegressionCV(random_state=RANDOM_STATE)
    cls.fit(X2, y)
    yp = cls.predict(X2)
else:
    cls = LogisticRegressionCV(random_state=RANDOM_STATE)
    cls.fit(X_dr, y)
    yp = cls.predict(X_dr)    
    if True:
        dr2 = TSNE(2,  init="pca", random_state=RANDOM_STATE)
    X2 = dr2.fit_transform(X_dr)

In [ ]:
from dos_clustering_misc import plot_X2_compare, assign_frequent_value
yp_freq = yp
plot_X2_compare(X2, y, yp_freq)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, yp_freq)
y_uniq = np.unique(y)
pd.DataFrame(cm, index=y_uniq, columns=y_uniq)


In [ ]:
from sklearn.metrics import classification_report
msg = classification_report(y,yp_freq)
print(msg)
